In [1]:
import pandas as pd
import matplotlib as mp

%matplotlib inline

In [ ]:
#Algunos datos de utilidad según conclusiones del TP1

#En qué horarios se realizan más subastas?
# Teniendo en cuenta solo los días hábiles
# 5~10    : < 200k
# 4,11~20 : 200k~300k
# 20~0   : 300k ~ 400k
# 1~2    : > 400k

# :. entre las 20~2 hs se realizan la mayor cantidad de subastas con picos entre 1~2 hs.

In [ ]:
# Depende un evento o una conversión de un usuario del día de la semana? Y de la hora?

#La cantidad de eventos se mantine constantes, excepto en los días M y X, donde encontramos un pico.
#Teniendo en cuenta la hora, la mayor cantidad de eventos se produce durante la tarde noche, inicio de la madrugada:
#Pico(+175k):    [0,1]
#Alta(150k~100k):(1,4)U[15,0)
#Media(100k~50k): [4,15)
#Baja(0~25k):(7,9)
#:. La mayor cantidad de eventos sucede durante el rango 15~1(tarde noche)

#INSTALACIONES
# La mayor cantidad se realiza los días M y X
# Los días M y X se duplica la cantidad de instalaciones aproximadamente(350~400)vs(750~800)
# Respecto del horario: Comportamiento similar a los eventos; la mayor cantidad se registran durante la tarde~noche(13~3)
# Afecta la presencia de wifi a la cantidad de instalaciones?
#El 80% de las instalaciones se lleva a cabo con dispositivos usando wifi.
#Cualés son las app mas populares?
# app_id | users | events
# 66       70k      325k
# 63       18k      181k 
# 145      17.5k    252k
# 64       17k      259k
# 103       15k     137k

In [4]:
auctions = pd.read_csv('../7506-tp2/data/auctions.csv.gzip',compression='gzip',\
                        usecols=['date','device_id','platform','ref_type_id','source_id'],\
                        dtype={'device_id':'int64','platform':'category','ref_type_id':'category','source_id':'category'},\
                        parse_dates=['date'])

In [5]:
auctions.head()

,date,device_id,platform,ref_type_id,source_id
0,2019-03-11 14:18:33.290763,6059599345986491085,1,1,0
1,2019-03-11 14:18:34.440157,1115994996230693426,1,1,0
2,2019-03-11 14:18:35.862360,7463856250762200923,1,1,0
3,2019-03-11 14:18:36.167163,7829815350267792040,1,1,0
4,2019-03-11 14:18:37.728590,1448534231953777480,1,1,0


## Predicciones
- En un instante dado, estimar St(d) el tiempo hasta que un dispositivo d aparezca de vuelta en una subasta RTB
- En un instante dado, estimar Sc(d) el tiempo hasta que un dispositivo d convierta



# Arboles - xgboost

In [6]:
import pandas as pd
import matplotlib as mp
import numpy as np
from datetime import timedelta
import datetime as dt
import gc

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb

%matplotlib inline

## Preparación de datos

## Configuracion

In [10]:
#Root de archivos de datos
root_dir = '../data/'
print('Root path data: '+root_dir)
#Lee menos data si es testing
is_testing = True
print('Es testing: '+str(is_testing))
fraccion_a_usar = 0.01
print('Faccion de registros a usar para testing: '+str(fraccion_a_usar))
# Cantidad minima de aparicion de equipos en un dataframe. Esta variable se usa para eliminar los registros
# que tengan pocas apariciones dentro del set de datos total.
min_devices = 10
print('Min de device a tener en cuenta: '+str(min_devices))

# Identificador unico de equipos
device_uuid = ['ref_type','ref_hash']

# Ventanas de tiempo
windows = pd.DataFrame({
'window_nr':[1,2,3,4],
'n': 1,
'min_date': dt.datetime(2019,4,18),
'max_date': [dt.datetime(2019,4,21),dt.datetime(2019,4,22),dt.datetime(2019,4,23),dt.datetime(2019,4,24)]
})

Root path data: ../data/
Es testing: True
Faccion de registros a usar para testing: 0.01
Min de device a tener en cuenta: 10


In [8]:
windows

,max_date,min_date,n,window_nr
0,2019-04-21,2019-04-18,1,1
1,2019-04-22,2019-04-18,1,2
2,2019-04-23,2019-04-18,1,3
3,2019-04-24,2019-04-18,1,4


In [9]:
#Se lee el dataframe de auctions

#Optimizado para menos memoria
auction_dtypes = {
    'ref_type_id': np.int8,
    'device_id': np.int64,
    'source_id': np.int8
}
#root_dir + 'auctions.csv.gzip'
auctions = pd.read_csv(root_dir+'auctions.csv.gzip',
                       compression = 'gzip',
                       dtype = auction_dtypes,
                       parse_dates = ['date'])
auctions.rename({'device_id':'ref_hash',
                 'ref_type_id':'ref_type'}, inplace=True, axis='columns')

# Para sumar los registros
auctions['n'] = 1

In [11]:
# Info de registros totales/a usar para entrenar
print('Total records:',auctions['n'].sum())
if is_testing:
    auctions = auctions.sample(frac=fraccion_a_usar)
    print("Is Testing, #records:", auctions['n'].sum())
else:
    print("Not Testing, #records:", auctions['n'].sum())

('Total records:', 47409528)
('Is Testing, #records:', 474095)


In [12]:
to_predict = pd.read_csv(root_dir+'target_competencia_ids.csv')
#predict reaparicion=st
predict_ids = to_predict.apply(lambda x: np.int64(x['ref_hash'][0:x['ref_hash'].find('_')]), axis='columns').drop_duplicates().to_frame()
#predict conversion=sc
predict_ids.head()

,0
0,1000169251625791246
2,1000395625957344683
4,1003027494996471685
6,1006670001679961544
8,1007573308966476713


### Arreglo de datos en auctions

In [13]:
auctions.head(3)

,date,ref_hash,ref_type,source_id,n
35941957,2019-04-25 12:54:18.427862,3645837532698533801,1,3,1
21062523,2019-04-22 04:52:18.337146,7019123669376337035,1,7,1
34331726,2019-04-21 22:13:06.496832,1458575463379897371,1,1,1


In [14]:
# Se eliminan los reg con pocas apariciones dentro del set de datos
print('Minimo de devices a usar '+str(min_devices))
orig_count = auctions['n'].sum()
auctions_clean = auctions.groupby(device_uuid, sort = False).filter(lambda x: x['n'].sum() >= min_devices) 
last_count = auctions_clean['n'].sum()
print('Cantidad auctions originales '+str(orig_count))
print('Cantidad auctions eliminados '+str(orig_count-last_count))
print('Cantidad auctions finales '+str(last_count))

Minimo de devices a usar 10
Cantidad auctions originales 474095
Cantidad auctions eliminados 287131
Cantidad auctions finales 186964


In [15]:
auctions_clean.head()

,date,ref_hash,ref_type,source_id,n
35941957,2019-04-25 12:54:18.427862,3645837532698533801,1,3,1
34331726,2019-04-21 22:13:06.496832,1458575463379897371,1,1,1
38927700,2019-04-20 18:01:00.661500,5516911236867393641,1,1,1
29486944,2019-04-23 07:51:04.852338,6496399488399992341,1,3,1
34669002,2019-04-24 04:11:12.219296,6811222176750924189,1,1,1


In [18]:
device_uuid+['date']

['ref_type', 'ref_hash', 'date']

In [22]:
auctions_clean.dtypes

date         datetime64[ns]
ref_hash              int64
ref_type               int8
source_id              int8
n                     int64
dtype: object

In [39]:
auc_clean = auctions_clean.sort_values(by=device_uuid+['date'], inplace=False)
#setea el próximo día de aparición(next_date)
auc_clean['next_date'] = auc_clean.groupby(device_uuid, as_index = False, sort=False)['date'].transform(lambda x: x.shift(-1))
# saco los registros 'borde', aquellos que no tienen 'next_date'
auc_clean = auc_clean.loc[(~auctions['next_date'].isnull())]
# Seteo los segundos de las apariciones
auc_clean['secs_to_next'] = (auc_clean['next_date'] - auc_clean['date']).transform(lambda x: round(x.total_seconds()))
auc_clean.head()

KeyError: 'next_date'

In [42]:
auc_clean['hs_to_next'] = (auc_clean['secs_to_next'] ).transform(lambda x: x/(60*60))
auc_clean.head()

,date,ref_hash,ref_type,source_id,n,next_date,secs_to_next,hs_to_next
11788129,2019-04-18 02:41:21.514671,2125435237741841,1,1,1,2019-04-18 07:46:13.565484,18292.0,5.081111
24648745,2019-04-18 07:46:13.565484,2125435237741841,1,3,1,2019-04-18 08:01:50.872944,937.0,0.260278
24545467,2019-04-18 08:01:50.872944,2125435237741841,1,1,1,2019-04-18 08:55:59.808056,3249.0,0.902500
28388057,2019-04-18 08:55:59.808056,2125435237741841,1,1,1,2019-04-18 08:57:49.989240,110.0,0.030556
24540533,2019-04-18 08:57:49.989240,2125435237741841,1,1,1,2019-04-18 09:08:45.657779,656.0,0.182222


In [46]:
auc_clean['n'].sum()

186964

In [47]:
auc_clean['date'].min()

Timestamp('2019-04-18 00:00:05.039826')

In [48]:
auc_clean['date'].max()

Timestamp('2019-04-26 23:59:57.384860')

# Armado de features

In [66]:
windows.set_index('window_nr')

,max_date,min_date,n
window_nr,,,
1,2019-04-21,2019-04-18,1
2,2019-04-22,2019-04-18,1
3,2019-04-23,2019-04-18,1
4,2019-04-24,2019-04-18,1


## Rearmado de ventanas

In [69]:
windows2 = pd.DataFrame({
'window_nr':[1,2,3,4],
'n': 1,
'min_date': [dt.datetime(2019,4,18),dt.datetime(2019,4,19),dt.datetime(2019,4,20),dt.datetime(2019,4,21)],
'max_date': [dt.datetime(2019,4,20),dt.datetime(2019,4,21),dt.datetime(2019,4,22),dt.datetime(2019,4,23)]
})
windows2

,max_date,min_date,n,window_nr
0,2019-04-20,2019-04-18,1,1
1,2019-04-21,2019-04-19,1,2
2,2019-04-22,2019-04-20,1,3
3,2019-04-23,2019-04-21,1,4


In [70]:
cutoff_date = windows.set_index('window_nr').loc[1,'max_date']
cutoff_date

Timestamp('2019-04-21 00:00:00')

In [74]:
train_auctions = auc_clean.loc[auctions['date'] < cutoff_date]
train_auctions.head(3)
train_auctions.count()

date            61251
ref_hash        61251
ref_type        61251
source_id       61251
n               61251
next_date       61105
secs_to_next    61105
hs_to_next      61105
dtype: int64

In [73]:
#Veo cuantos device tengo para el 'train_auctions'
devices = train_auctions[device_uuid].drop_duplicates()
devices['ref_hash'].count()

8218

In [99]:
#train_auctions[train_auctions['ref_hash'] == 2125435237741841]#13
train_auctions.groupby(by='ref_hash')['secs_to_next'].sum().loc[2125435237741841,]/13#344042.0

26464.76923076923

In [117]:
train_auctions.head()

,date,ref_hash,ref_type,source_id,n,next_date,secs_to_next,hs_to_next
11788129,2019-04-18 02:41:21.514671,2125435237741841,1,1,1,2019-04-18 07:46:13.565484,18292.0,5.081111
24648745,2019-04-18 07:46:13.565484,2125435237741841,1,3,1,2019-04-18 08:01:50.872944,937.0,0.260278
24545467,2019-04-18 08:01:50.872944,2125435237741841,1,1,1,2019-04-18 08:55:59.808056,3249.0,0.902500
28388057,2019-04-18 08:55:59.808056,2125435237741841,1,1,1,2019-04-18 08:57:49.989240,110.0,0.030556
24540533,2019-04-18 08:57:49.989240,2125435237741841,1,1,1,2019-04-18 09:08:45.657779,656.0,0.182222


In [120]:
#Se desea saber el tiempo promedio entre arribos de los dispositivos a las encuestas.
temp = train_auctions.groupby(device_uuid, as_index='False')['secs_to_next'].mean().to_frame()['secs_to_next']
temp.columns = ['secs_to_next_mean']

In [141]:
train_auc = train_auctions.merge(temp,how='inner',on='ref_hash')
train_auc
train_auc = train_auc.rename(index=str,columns= {"secs_to_next_x":"secs_to_next",'secs_to_next_y':'secs_to_next_mean'})
train_auc.head()

,date,ref_hash,ref_type,source_id,n,next_date,secs_to_next,hs_to_next,secs_to_next_mean
0,2019-04-18 02:41:21.514671,2125435237741841,1,1,1,2019-04-18 07:46:13.565484,18292.0,5.081111,26464.769231
1,2019-04-18 07:46:13.565484,2125435237741841,1,3,1,2019-04-18 08:01:50.872944,937.0,0.260278,26464.769231
2,2019-04-18 08:01:50.872944,2125435237741841,1,1,1,2019-04-18 08:55:59.808056,3249.0,0.902500,26464.769231
3,2019-04-18 08:55:59.808056,2125435237741841,1,1,1,2019-04-18 08:57:49.989240,110.0,0.030556,26464.769231
4,2019-04-18 08:57:49.989240,2125435237741841,1,1,1,2019-04-18 09:08:45.657779,656.0,0.182222,26464.769231


# Primer prueba con XGBoost